In [1]:
import pandas as pd
import gurobi_logtools as glt
from report_functions import *

In [25]:
samp_100_dyn_df = pd.read_csv("/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/sample_tests/2024_02_29medium_new_equip_100samples/dynamic_linear/ld_medium_instance_new_config_S4_results.csv")
samp_50_dyn_df = pd.read_csv("/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/sample_tests/2024_02_29medium_new_equip_50samples/dynamic_linear/ld_medium_instance_new_config_S4_results.csv")
samp_50_dyn_df.head(100)


# def add_log_and_gap(df, log_file_location, file_name_stuff):
#     df = read_csv_add_log( log_file_location, file_name_stuff, df)
#     df = read_csv_add_gap(df, gapname="gap_dyn")
#     return df

# samp_100_dyn_df = add_log_and_gap(samp_100_dyn_df, 
#                                   "/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/sample_tests/2024_02_29medium_new_equip_50samples/dynamic_linear/", 
#                                   "ld_medium_instance_new_config_S40")
log_file_location_50 = "/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/sample_tests/2024_02_29medium_new_equip_50samples/dynamic_linear/"
file_name_stuff_50 = "ld_medium_instance_new_config_S4"
log_file_location_100 = "/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/sample_tests/2024_02_29medium_new_equip_100samples/dynamic_linear/"
file_name_stuff_100 = "ld_medium_instance_new_config_S4"
def add_log_and_gap(log_file_location, file_name_stuff, df):
    df = read_csv_add_log( log_file_location, file_name_stuff, df)
    df = read_csv_add_gap(df, gapname="gap_dyn")
    return df
samp_100_dyn_df = add_log_and_gap(log_file_location_100, file_name_stuff_100, samp_100_dyn_df)
samp_50_dyn_df = add_log_and_gap(log_file_location_50, file_name_stuff_50, samp_50_dyn_df)
samp_100_dyn_df.head(100)


,Unnamed: 0,model_name,instance_name,obj_value,solver_status,equipment_instance,xp_config_file,model_file,variables_folder,run_time,group_counter,log,gap_dyn
0,0,dynamic linear labour recourse,n=50_131_n=50_132,13405.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4247.475657,0,/Users/letshopethisworks2/Documents/Phd Paper ...,46.7363
1,1,dynamic linear labour recourse,n=50_12_n=50_13,7487.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4307.290886,1,/Users/letshopethisworks2/Documents/Phd Paper ...,45.4788
2,2,dynamic linear labour recourse,n=50_14_n=50_15,13127.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4247.077502,2,/Users/letshopethisworks2/Documents/Phd Paper ...,64.4016
3,3,dynamic linear labour recourse,n=50_19_n=50_20,6828.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4321.156084,3,/Users/letshopethisworks2/Documents/Phd Paper ...,26.0838
4,4,dynamic linear labour recourse,n=50_1_n=50_2_n=50_3,8688.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4243.335348,4,/Users/letshopethisworks2/Documents/Phd Paper ...,49.0331
5,5,dynamic linear labour recourse,n=50_9_n=50_10_n=50_11,11685.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4243.043825,5,/Users/letshopethisworks2/Documents/Phd Paper ...,61.7629
6,6,dynamic linear labour recourse,n=50_21_n=50_22_n=50_23,9715.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4241.828736,6,/Users/letshopethisworks2/Documents/Phd Paper ...,56.1194
7,7,dynamic linear labour recourse,n=50_53_n=50_54_n=50_55,14049.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4242.116869,7,/Users/letshopethisworks2/Documents/Phd Paper ...,50.8719
8,8,dynamic linear labour recourse,n=50_54_n=50_55_n=50_56,14123.0,1,SALBP_benchmark/MM_instances/equipment_data/ra...,SALBP_benchmark/MM_instances/medium_instance_n...,SALBP_benchmark/MM_instances/model_data/medium...,model_runs/2024_02_29medium_new_equip_100sampl...,4243.991755,8,/Users/letshopethisworks2/Documents/Phd Paper ...,54.6272


## Processing MC results

In [3]:
#Reads all of the .log files in /Users/letshopethisworks2/Documents/Phd Paper material/model_runs/mc_tests/2024_03_11mc_test_40samples. The log files are in folders
#with numbers for names
import os
import re

def get_log(log_file_location):
    #look for .log files in all folders in the log_file_location
    log_files = []
    for root, dirs, files in os.walk(log_file_location):
        for file in files:
            if file.endswith(".log"):
                log_files.append(os.path.join(root, file))
    return log_files

log_file_location = "/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/mc_tests/2024_03_11mc_test_40samples"
log_files = get_log(log_file_location)
print(len(log_files))

110


In [32]:

#Gets the row of the csv run file from the file path. 
def get_row_from_file_path(file_path, row_location = 8):
    #gets the row of the csv run file from the file path. 
    file_path = file_path.split("/")
    file_path = file_path[row_location]
    return file_path


def summarize_logs(log_files, csv_config_file):
    results = glt.parse(log_files)
    summary = results.summary(results)
    #gets column names of summary table
    print(summary.columns)
    #Only keeps objVal, MIPGap, LogFilePath
    summary = summary[['ObjVal', 'MIPGap', 'LogFilePath']]
    summary['row'] = summary['LogFilePath'].apply(lambda x: int(get_row_from_file_path(x)))
    summary = summary.sort_values(by='row')
    summary.reset_index(drop=True, inplace=True)
    #groups the summary table by every 10 rows, takes the average and standard deviation of the ObjVal and MIPGap
    config_df = pd.read_csv(csv_config_file)
    #merges the summary table with the config file, using the index of the config file and the row of the summary table
    summary = pd.merge( config_df,summary, left_index=True, right_on='row')
    #Groups by the model yaml
    summary = summary.groupby('model_yaml').agg({'ObjVal': ['mean', 'std'], 'MIPGap': ['mean', 'std']})
    # summary['group'] = summary.index // instance_repeat
    # summary = summary.groupby('group').agg({'ObjVal': ['mean', 'std'], 'MIPGap': ['mean', 'std']})
    return summary


summary = summarize_logs(log_files, "../SALBP_benchmark/MM_instances/csv_config/no_tree.csv" )
summary.head(10)

Index(['Platform', 'Time', 'ReadingTime', 'ModelName', 'Rows', 'Columns',
       'Nonzeros', 'PhysicalCores', 'LogicalProcessors', 'Threads',
       'NumConstrs', 'NumVars', 'NumNZs', 'Fingerprint', 'PresolvedNumConVars',
       'PresolvedNumIntVars', 'PresolvedNumBinVars', 'MinCoeff', 'MaxCoeff',
       'MinObjCoeff', 'MaxObjCoeff', 'MinBound', 'MaxBound', 'MinRHS',
       'MaxRHS', 'PresolveTime', 'PresolvedNumConstrs', 'PresolvedNumVars',
       'PresolvedNumNZs', 'RelaxObj', 'RelaxIterCount', 'RelaxTime',
       'NodeCount', 'IterCount', 'Runtime', 'Work', 'ObjVal', 'ObjBound',
       'MIPGap', 'Cuts: Cover', 'Cuts: Implied bound', 'Cuts: Clique',
       'Cuts: MIR', 'Cuts: StrongCG', 'Cuts: GUB cover', 'Cores', 'SolCount',
       'Status', 'ModelType', 'ChangedParams', 'LogFilePath', 'LogNumber',
       'Cuts: Zero half', 'Cuts: RLT', 'Cuts: Relax-and-lift',
       'Cuts: Inf proof', 'Cuts: BQP', 'Cuts: Gomory', 'Cuts: PSD',
       'TokenServer (Parameter)', 'TimeLimit (Parameter)

ObjVal              \
                                                       mean         std   
model_yaml                                                                
SALBP_benchmark/MM_instances/model_data/small_i...   4972.0    0.000000   
SALBP_benchmark/MM_instances/model_data/small_i...   4972.0    0.000000   
SALBP_benchmark/MM_instances/model_data/small_i...  11178.3  255.201467   
SALBP_benchmark/MM_instances/model_data/small_i...  10702.2  159.613979   
SALBP_benchmark/MM_instances/model_data/small_i...  11534.8  217.942296   
SALBP_benchmark/MM_instances/model_data/small_i...  12032.1  399.185963   
SALBP_benchmark/MM_instances/model_data/small_i...  12134.5  361.874211   
SALBP_benchmark/MM_instances/model_data/small_i...  12354.8  222.266507   
SALBP_benchmark/MM_instances/model_data/small_i...  12013.6  156.153201   
SALBP_benchmark/MM_instances/model_data/small_i...  12776.9  180.641726   

                                                      MIPGap            
                                                        mean       std  
model_yaml                                                              
SALBP_benchmark/MM_instances/model_data/small_i...  0.000000  0.000000  
SALBP_benchmark/MM_instances/model_data/small_i...  0.000000  0.000000  
SALBP_benchmark/MM_instances/model_data/small_i...  0.076667  0.045025  
SALBP_benchmark/MM_instances/model_data/small_i...  0.057389  0.058697  
SALBP_benchmark/MM_instances/model_data/small_i...  0.134956  0.039449  
SALBP_benchmark/MM_instances/model_data/small_i...  0.151745  0.057076  
SALBP_benchmark/MM_instances/model_data/small_i...  0.168899  0.044655  
SALBP_benchmark/MM_instances/model_data/small_i...  0.166424  0.022614  
SALBP_benchmark/MM_instances/model_data/small_i...  0.142628  0.041048  
SALBP_benchmark/MM_instances/model_data/small_i...  0.157997  0.029017

In [24]:
#/Users/letshopethisworks2/Documents/Phd Paper material/model_runs/mc_tests/2024_03_11mc_test_40samples/0/no_tree_row_0/dynamic_linear/ld_small_instance_longSeq0.log





ObjVal                MIPGap          
          mean         std      mean       std
group                                         
0       4972.0    0.000000  0.000000  0.000000
1       4972.0    0.000000  0.000000  0.000000
2      11178.3  255.201467  0.076667  0.045025
3      10702.2  159.613979  0.057389  0.058697
4      11534.8  217.942296  0.134956  0.039449
5      12032.1  399.185963  0.151745  0.057076
6      12134.5  361.874211  0.168899  0.044655
7      12354.8  222.266507  0.166424  0.022614
8      12013.6  156.153201  0.142628  0.041048
9      12776.9  180.641726  0.157997  0.029017
10     12247.5  274.006590  0.165641  0.052151

In [29]:
config_df = pd.read_csv("../SALBP_benchmark/MM_instances/csv_config/no_tree.csv")
config_df.head(10)

,model_yaml,equipment_yaml,scenario_tree_yaml,config_yaml
0,SALBP_benchmark/MM_instances/model_data/small_...,SALBP_benchmark/MM_instances/equipment_data/ra...,No Tree,SALBP_benchmark/MM_instances/small_instance_lo...
1,SALBP_benchmark/MM_instances/model_data/small_...,SALBP_benchmark/MM_instances/equipment_data/ra...,No Tree,SALBP_benchmark/MM_instances/small_instance_lo...
2,SALBP_benchmark/MM_instances/model_data/small_...,SALBP_benchmark/MM_instances/equipment_data/ra...,No Tree,SALBP_benchmark/MM_instances/small_instance_lo...
3,SALBP_benchmark/MM_instances/model_data/small_...,SALBP_benchmark/MM_instances/equipment_data/ra...,No Tree,SALBP_benchmark/MM_instances/small_instance_lo...
4,SALBP_benchmark/MM_instances/model_data/small_...,SALBP_benchmark/MM_instances/equipment_data/ra...,No Tree,SALBP_benchmark/MM_instances/small_instance_lo...
5,SALBP_benchmark/MM_instances/model_data/small_...,SALBP_benchmark/MM_instances/equipment_data/ra...,No Tree,SALBP_benchmark/MM_instances/small_instance_lo...
6,SALBP_benchmark/MM_instances/model_data/small_...,SALBP_benchmark/MM_instances/equipment_data/ra...,No Tree,SALBP_benchmark/MM_instances/small_instance_lo...
7,SALBP_benchmark/MM_instances/model_data/small_...,SALBP_benchmark/MM_instances/equipment_data/ra...,No Tree,SALBP_benchmark/MM_instances/small_instance_lo...
8,SALBP_benchmark/MM_instances/model_data/small_...,SALBP_benchmark/MM_instances/equipment_data/ra...,No Tree,SALBP_benchmark/MM_instances/small_instance_lo...
9,SALBP_benchmark/MM_instances/model_data/small_...,SALBP_benchmark/MM_instances/equipment_data/ra...,No Tree,SALBP_benchmark/MM_instances/small_instance_lo...
